# Shifting Db2 with an Upgrade
This lab will clone a Db2 Columnar database from Version 11.1 to Version 11.5 in a Pod running on the Kubernetes cluster. The information required for the shift operation is identical to the first shift lab except that we are now upgrading the database during the process.

## Terminal Window
Before running the examples in this lab, you will need to open up a terminal window. If you have not already done so, see the <a href="./Terminal.ipynb">Opening up a Terminal Window</a> section. 

The commands that you need to copy and execute in the terminal window are surrounded with a blue box. Select all the text in the box and then use the copy keys (^C or &#x2318;C) to place the text into the clipboard.

<div class="boxed">The command you need to copy and paste into the terminal window are found in here.
</div>

When you switch to the terminal window, you will use the appropriate Paste command (^&#x21E7;v or &#x2318;v). This will load the command into the terminal window and immediately execute it.

## Db2inst User on Server 2
For this lab we are using the db2inst2 userid on server 1. This is a Db2 11.1 instance and contains one columnar database that we want to shift into Db2 on Kubernetes. First we will make sure that we are disconnected from `db2inst1` and are running as the base `k8s` user.

<div class="boxed">[ $(whoami) != 'k8s' ] && exit</div>

Next we connect as `db2inst2` on server 1 (`10.0.0.1`)

<div class="boxed">ssh db2inst2@10.0.0.1</div>

We also will double check that we can connect to the Kubernetes server and that our database exists on the system.

<div class="boxed">clear
kubectl get pods -n db2u
db2 list db directory
</div>

Your output should look similar to the following:
```
NAME                                    READY   STATUS      RESTARTS   AGE
c-demo-instdb-dnt74                     0/1     Completed   0          2d18h
c-demo-restore-morph-4phbd              0/1     Completed   0          2d18h
db2u-operator-manager-c849d76c4-zvdsm   1/1     Running     2          2d18h
c-demo-etcd-0                           1/1     Running     2          2d18h
c-demo-db2u-0                           1/1     Running     2          2d18h
[db2inst2@db2source ~]$ db2 list db directory

 System Database Directory

 Number of entries in the directory = 1

Database 1 entry:

 Database alias                       = CQWCOL
 Database name                        = CQWCOL
 Local database directory             = /home/db2inst2
 Database release level               = 14.00
 Comment                              =
 Directory entry type                 = Indirect
 Catalog database partition number    = 0
 Alternate server hostname            =
 Alternate server port number         =
 ```

## Upgrade Db2 During Shift

During the clone process, the Db2 Shift command will determine whether the Source database will need to be upgraded when it reaches the target Pod or Instance. The upgrade process is done automatically, with no additional settings required. Any database migration from 10.5 or 11.1 requires that the database be in an offline state for the migration to occur successfully.

Let's check the number of rows that we are moving from the largest table in the CQWCOL database.

<div class="boxed">clear
db2 connect to cqwcol
db2 "select count(*) from customer"
</div>

There should be 2 million rows. This database is a subset of the 100Gb CQW workload and is about 2Gb in size after being loaded as columnar data.

## Identify the Source database
For this lab we will be using the only database that db2inst2@server1 has. This is the `CQWCOL` database which is approximately 2Gb in size. This database is a Version 11.1 database so it will need to go through a migration to get it up to the 11.5 level that is in the POD.
Note: The instance owner is `db2inst2`!

## Instance Ownership

Whenever a database is being shifted to another location, you must ensure that the target location is able to access the database. All Db2U pods use `db2inst1` as the instance owner. You are not able to add new users using traditional operating system commands (authentication is via LDAP or similar security mechanisms). To ensure that the target location can access the contents of the database, a `db2inst1` user must exist on the source database.

Since the instance owner is **not** `db2inst1`, you must add this userid to the database using the following commands:

<div class="boxed">clear
db2 connect to cqwcol
db2 GRANT SECADM ON DATABASE TO USER db2inst1
db2 GRANT DBADM ON DATABASE TO USER db2inst1
</div>

Once these privileges have been granted to `db2inst1`, the target location will be able to access the database and manage it. If this step is not done, it will be difficult to access the tables without setting up LDAP.  Note that the `db2inst1` userid does not need to exist on the source system. Only when the database moves to the target system does this userid become valid.

## Gather Shift Details
The scenario that we are running is Shift a Db2 database to a POD. This is the same process as found in a previous lab. You will need the following information. 
* Name of the local database (`CQWCOL`)
* Instance owner (`db2inst2`)
* Target Location (`Kubernetes`)
* The target database name (`db2oltp`)
* POD Project (`db2u`)
* POD Name (`c-demo-db2u-0`)

The target database is named `db2oltp` (it doesn't change even though we do multiple shifts to this database). The POD and project information are the same:

* Project: `db2u`
* POD Name: `c-demo-db2u-0`

## Start Db2 Shift
At this point you are ready to analyze the Db2 11.1 database. Switch to the c2c directory for user db2inst2@server1 and start the Db2 Shift command in the Db2 Shift Window.

<div class="boxed">clear
cd ~/c2c
./db2shift --shiftpod 
</div>

![C2C Shift Pod](images/c2c-shiftpod.jpg)

Update the information on this screen with the values you collected earier.
* Database Name: `CQWCOL`
* Instance Owner: `db2inst2` <-- **Check that use use db2inst2!**
* Target location: `Kubernetes`
* Target Database: `db2oltp`
* POD Project: `db2u`
* POD Name: `c-demo-db2u-0`

The following settings will not be used during this lab and should be set to the default values.
* Metadata: `Refresh`
* Sync: `None`
* Move Options: `Database`
* Exclude Routines: `OFF`
* Database Mode: `Online`
* Thread Count: `4`
* Compression: `0`
* Overrides: `None`

## Analyze the Source and Target
Up to this point in the lab we have not considered the instance settings at the target. There is the possibility that the database you are shifting may require certain settings at the instance level to operate properly.

For each Shift and Clone scenario, the Db2 Shift program provides an analysis option (^A) that will check the source and target databases and provide a summary of differences that you need to be aware of. Before the database is moved, an analysis step will be performed. Make sure that the Metadata settings are set to `Refresh` when running this step.
 
Once you have filled in all the values for the shift, press ^A to begin the analysis step. When the analysis completes you will be presented with a summary of instance settings that are different between the source and target.

![Analysis](images/c2c-analysis.jpg)

The Db2 Shift program has detected that your source database will need to be upgraded during the shift process and that there are three settings that are different at the target instance. Two of the settings may need to be reviewed:
* instance_memory – The amount of memory the instance is using
* intra_parallel – Is parallelism allowed within the system

If you are not familiar with what a setting is used for, use the cursor keys to move to the parameter and press ^F for field help.

![Analysis](images/c2c-analysishelp.jpg)

### Intra_Parallel Setting
The one setting that is critical in the list is the `intra_parallel` setting which is `ON` on our source system. 
If you do not see `intra_parallel` in the list of settings, you can continue onto the Gather Metadata step.

A previous lab should have updated this value already, but if you see it on this list you should select it to be updated during the shift. Use your cursor keys to move to the line with the `intra_parallel` field and press the space bar to select it (the circle should fill in and the line will turn light blue).

Now press ^X (CTRL-X) to generate overrides. This will populate the original deploy screen with a setting in the `Overrides` field.

![Overrides](images/c2c-overrides.jpg)

The overrides section of the form now contains the updated `intra_parallel` setting that is required at the destination instance. 

## Gather Metadata Only
As mentioned earlier, you must make sure that a database that is being upgraded be **OFFLINE** for the migration to be successful. Before we can shut down the database, we must gather the control information required for the Db2 Shift command. The Db2 Shift command will not be able to run without the metadata being available for the source database.
The Metadata option is used to gather all the control information required to perform a shift. 
 
The Db2 Shift command generates metadata that is used during the shift process. This metadata is key to determining which objects need to be moved from a source to destination as well as validating that the source can be successfully moved.
 
The Settings Only option will generate the metadata files only and not execute the shift. The benefit of running the Shift with Settings Only is that the source and destination will be checked for connectivity so that you will know whether the shift will be able to run.

![Metadata](images/c2c-settings.jpg)

Change the Metadata to Settings Only and Press ^X to check that the parameters are correct and review the syntax that is generated. Now that all checks have passed, execute the command (^X).
 
The results screen will display that the settings have been generated. 

![Metadata](images/c2c-settings-done.jpg)

At this point we can shut our database down. Escape out of the Db2 Shift program back to the command line and issue the following commands.

<div class="boxed">clear
db2 connect reset
db2stop force
</div>

Start up the Db2 Shift program again.

<div class="boxed">clear
cd ~/c2c
./db2shift --shiftpod 
</div>

All of the details should still be there from the last run. Change the `Metadata` field to `None`.

![No Metadata](images/c2c-metadata-none.jpg)
 
When `None` is selected, the Db2 Shift command will not attempt to connect to the source or destination to gather the control information. Instead, the program will use the settings files that were generated previously and are found in the current working directory. You can view these files to review what control information Db2 Shift gathers from the source and destination databases:
* source.settings – Database information at the source
* destination.settings – Database information at the destination

## Running the Shift
 
Db2 Shift provides two options when dealing with the state of a database during the shift process:
* --online – database is online while the shift is taking place
* --offline – database has been shut down

The offline mode must be used when shifting a database that requires a migration from an older release of Db2. This option must be specified when shifting Db2 versions 10.5 or 11.1. 

![Offline](images/c2c-online.jpg)

Make sure to change the value to `offline`. Change the Thead count to `8` to provide more progress information while the shift runs. 

Once you have updated the values, press the ^X key to review the command and then ^X to start the shift. When the shift is complete, you can restart the local database and examine the contents of the remote POD. 

**Note:** This will take approximately 5 minutes to complete the shift of the data. It may appear that the copy is stalling because the progress meter isn't moving. The progress meter is tied to the database objects being moved and not the data volume.

When the shift is complete, exit Db2 Shift and restart the database.

<div class="boxed">db2start</div>

**Note:** If you had to update the instance with the `intra_parallel` setting, you will need to restart the Db2u instance for the configuration changes to take effect. If you didn't restart the instance, the query would not work, and you would have received an SQL error code.
```
SQL1668N The operation failed because the operation is not supported with this environment. Reason code: 5
```

<div class="boxed">clear
kubectl exec -i c-demo-db2u-0 -n db2u -- bash << EOF
su - db2inst1 "db2stop force"
EOF

Give the database about a minute to restart automatically. Next, we view the contents of the POD that we copied into. Note that the owner of the table is still `db2inst2`.

<div class="boxed">clear
kubectl exec -i c-demo-db2u-0 -n db2u -- bash << EOF
cat << EOT > /tmp/db2cmd.db2
CONNECT TO DB2OLTP;
SELECT COUNT(*) FROM DB2INST2.CUSTOMER;
QUIT;
EOT
su - db2inst1 "db2 -tvf /tmp/db2cmd.db2"
EOF
</div>

## Summary
In this lab you:
* Learned how to shift and upgrade a Db2 11.1 database
* Analyze a database to determine if settings needed updating
* Shift the database while it is offline by generating settings in an earlier step

#### Credits: IBM 2024, George Baklarz [baklarz@ca.ibm.com]